In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout
import matplotlib.pyplot as plt
from util import plot_history
import feature_builder
from transformers import BertTokenizer
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import TFBertModel
import tensorflow_hub as hub

Cargo datasets

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

Armo features con el feature_builder

In [3]:
#X_train = prepare_for_cnn(train_df)
#y_train = train_df['target'].values

Creo el modelo y corro la CNN

Calculemos el test para kaggle

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)

In [ ]:
def bert_encode(data,maximum_length) :
  input_ids = []
  attention_masks = []
  

  for i in range(len(data.text)):
      encoded = tokenizer.encode_plus(
        
        data.text[i],
        add_special_tokens=True,
        max_length=maximum_length,
        pad_to_max_length=True,
        
        return_attention_mask=True,
        
      )
      
      input_ids.append(encoded['input_ids'])
      attention_masks.append(encoded['attention_mask'])
  return np.array(input_ids),np.array(attention_masks)

In [ ]:
max_length = max([len(x) for x in train_df.text])
train_input_ids, train_attention_masks = bert_encode(train_df, max_length)

In [ ]:
def create_model(bert_model):
  input_ids = tf.keras.Input(shape=(max_length, ),dtype='int32')
  attention_masks = tf.keras.Input(shape=(max_length, ),dtype='int32')
  
  output = bert_model([input_ids,attention_masks])
  output = output[1]
  output = tf.keras.layers.Dense(32,activation='relu')(output)
  output = tf.keras.layers.Dropout(0.2)(output)

  output = tf.keras.layers.Dense(1,activation='sigmoid')(output)
  model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
  model.compile(Adam(lr=6e-6), loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
bert_model = TFBertModel.from_pretrained('bert-large-uncased')
model = create_model(bert_model)
model.summary()

In [ ]:
# filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
# checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]
# add callbacks = callbacks_list to model.fit

In [ ]:
history = model.fit([train_input_ids, train_attention_masks], train_df.target, validation_split=0.2, epochs=2)
util.plot_history(history)

In [ ]:
ids = test_df['id']

final_test = prepare_for_cnn(test_df)
prediction = model.predict_classes(final_test)

final_df = pd.DataFrame({'target': [x[0] for x in prediction]}, index=ids)
final_df

In [ ]:
final_df['target'].value_counts()

In [ ]:
final_df.count()

In [ ]:
final_df.to_csv('cnn.csv')